In [1]:
import cv2
import numpy as np
from tkinter import Tk, Button, Scale, HORIZONTAL, filedialog, Toplevel, colorchooser, Canvas, Frame, Scrollbar, VERTICAL, RIGHT, LEFT, Y, BOTH, Label
from PIL import Image, ImageTk

class ImageEditor:
    def __init__(self, master):
        self.master = master
        self.master.title("簡易圖像編輯器")

        self.original_image = None
        self.processed_image = None
        self.image_states = []
        self.undo_stack = []

        self.drawing = False
        self.brush_size = 5
        self.brush_color = (255, 0, 0)

        # Title Label
        self.title_label = Label(master, text="圖像編輯程式", font=("Arial", 25))
        self.title_label.pack()

        # Frame and canvas for the image
        self.frame = Frame(master, width=800, height=600)
        self.frame.pack(expand=True, fill=BOTH)

        self.canvas = Canvas(self.frame, bg='white')
        self.scroll_y = Scrollbar(self.frame, orient=VERTICAL, command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scroll_y.set)

        self.scroll_y.pack(side=RIGHT, fill=Y)
        self.canvas.pack(side=LEFT, expand=True, fill=BOTH)

        self.image_on_canvas = None

        self.load_button = Button(master, text="載入圖像", command=self.load_image)
        self.load_button.pack()

        self.invert_button = Button(master, text="反轉圖像", command=self.invert_image)
        self.invert_button.pack()

        self.brightness_scale = Scale(master, from_=-127, to=127, orient=HORIZONTAL, label="亮度", command=self.update_image)
        self.brightness_scale.pack()

        self.contrast_scale = Scale(master, from_=-127, to_127, orient=HORIZONTAL, label="對比度", command=self.update_image)
        self.contrast_scale.pack()

        self.saturation_scale = Scale(master, from_=-100, to_100, orient=HORIZONTAL, label="飽和度", command=self.update_image)
        self.saturation_scale.pack()

        self.crop_button = Button(master, text="裁剪圖像", command=self.start_crop)
        self.crop_button.pack()

        self.rotate_button = Button(master, text="旋轉圖像", command=self.rotate_image)
        self.rotate_button.pack()

        self.brush_button = Button(master, text="畫筆", command=self.prepare_brush)
        self.brush_button.pack()

        self.undo_button = Button(master, text="撤銷", command=self.undo)
        self.undo_button.pack()

        self.redo_button = Button(master, text="重做", command=self.redo)
        self.redo_button.pack()

        self.save_button = Button(master, text="保存圖像", command=self.save_image)
        self.save_button.pack()

        self.grayscale_button = Button(master, text="灰階", command=self.apply_grayscale)
        self.grayscale_button.pack()

        self.sepia_button = Button(master, text="復古", command=self.apply_sepia)
        self.sepia_button.pack()

        self.edge_button = Button(master, text="邊緣檢測", command=self.apply_edge_detection)
        self.edge_button.pack()

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp")])
        if file_path:
            self.original_image = cv2.imread(file_path)
            if self.original_image is not None:
                self.processed_image = self.original_image.copy()
                self.image_states = [self.processed_image.copy()]
                self.undo_stack.clear()
                self.show_image(self.original_image)
            else:
                print("Failed to load image. Please select a valid image file.")

    def save_image(self):
        if self.processed_image is not None:
            file_path = filedialog.asksaveasfilename(defaultextension=".jpg",
                                                     filetypes=[("JPEG files", "*.jpg"),
                                                                ("PNG files", "*.png"),
                                                                ("BMP files", "*.bmp")])
            if file_path:
                cv2.imwrite(file_path, self.processed_image)

    def invert_image(self):
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.bitwise_not(self.processed_image)
            self.show_image(self.processed_image)

    def update_image(self, _=None):
        if self.original_image is not None:
            brightness = self.brightness_scale.get()
            contrast = self.contrast_scale.get()
            saturation = self.saturation_scale.get()
            self.processed_image = self.adjust_brightness_contrast(self.original_image, brightness, contrast)
            self.processed_image = self.adjust_saturation(self.processed_image, saturation)
            self.show_image(self.processed_image)

    def adjust_brightness_contrast(self, image, brightness=0, contrast=0):
        brightness = np.clip(brightness, -127, 127)
        contrast = np.clip(contrast, -127, 127)

        B = brightness / 127.0
        C = contrast / 127.0

        k = np.tan((45 + 44 * C) / 180 * np.pi)

        img = image.astype(np.float32)
        img = (img - 127.5 * (1 - B)) * k + 127.5 * (1 + B)
        img = np.clip(img, 0, 255).astype(np.uint8)

        return img

    def adjust_saturation(self, image, saturation=0):
        saturation = np.clip(saturation, -100, 100)
        if saturation == 0:
            return image

        img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
        img[:, :, 1] = img[:, :, 1] * (1 + saturation / 100.0)
        img[:, :, 1] = np.clip(img[:, :, 1], 0, 255)
        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_HSV2BGR)

        return img

    def start_crop(self):
        if self.processed_image is not None:
            self.cropping = True
            self.crop_rect = [0, 0, 0, 0]
            cv2.namedWindow("Crop Image")
            cv2.setMouseCallback("Crop Image", self.mouse_crop)
            while self.cropping:
                img_copy = self.processed_image.copy()
                if self.crop_rect[2] != 0 and self.crop_rect[3] != 0:
                    cv2.rectangle(img_copy, (self.crop_rect[0], self.crop_rect[1]), 
                                  (self.crop_rect[2], self.crop_rect[3]), (0, 255, 0), 2)
                cv2.imshow("Crop Image", img_copy)
                cv2.waitKey(1)
            cv2.destroyWindow("Crop Image")
            self.crop_image()

    def mouse_crop(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.crop_rect[0] = x
            self.crop_rect[1] = y
        elif event == cv2.EVENT_LBUTTONUP:
            self.crop_rect[2] = x
            self.crop_rect[3] = y
            self.cropping = False

    def crop_image(self):
        if self.processed_image is not None:
            x1, y1, x2, y2 = self.crop_rect
            if x1 < x2 and y1 < y2:
                self.save_state()
                self.processed_image = self.processed_image[y1:y2, x1:x2]
                self.show_image(self.processed_image)
            else:
                print("Invalid crop coordinates")

    def rotate_image(self):
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.rotate(self.processed_image, cv2.ROTATE_90_CLOCKWISE)
            self.show_image(self.processed_image)

    def prepare_brush(self):
        if self.processed_image is not None:
            self.brush_window = Toplevel(self.master)
            self.brush_window.title("Brush Tool")

            self.brush_size_scale = Scale(self.brush_window, from_=1, to=50, orient=HORIZONTAL, label="Brush Size")
            self.brush_size_scale.pack()

            self.color_button = Button(self.brush_window, text="Choose Color", command=self.choose_color)
            self.color_button.pack()

            self.start_drawing_button = Button(self.brush_window, text="Start Drawing", command=self.start_brush)
            self.start_drawing_button.pack()

            self.brush_window.transient(self.master)
            self.brush_window.grab_set()

    def choose_color(self):
        color_code = colorchooser.askcolor(title="選擇畫筆顏色")
        if color_code:
            self.brush_color = (int(color_code[0][2]), int(color_code[0][1]), int(color_code[0][0]))

    def start_brush(self):
        self.brush_size = self.brush_size_scale.get()
        self.brush_window.destroy()

        if self.processed_image is not None:
            self.drawing = False

            cv2.namedWindow("Draw Image")
            cv2.setMouseCallback("Draw Image", self.mouse_draw)
            while True:
                cv2.imshow("Draw Image", self.processed_image)
                if cv2.waitKey(1) & 0xFF == 27:
                    break
            cv2.destroyWindow("Draw Image")
            self.save_state()
            self.show_image(self.processed_image)

    def mouse_draw(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                cv2.circle(self.processed_image, (x, y), self.brush_size, self.brush_color, -1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False

    def save_state(self):
        if self.processed_image is not None:
            self.image_states.append(self.processed_image.copy())
            self.undo_stack.clear()

    def undo(self):
        if len(self.image_states) > 1:
            self.undo_stack.append(self.image_states.pop())
            self.processed_image = self.image_states[-1]
            self.show_image(self.processed_image)

    def redo(self):
        if self.undo_stack:
            self.image_states.append(self.undo_stack.pop())
            self.processed_image = self.image_states[-1]
            self.show_image(self.processed_image)

    def apply_grayscale(self):
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2GRAY)
            self.processed_image = cv2.cvtColor(self.processed_image, cv2.COLOR_GRAY2BGR)  # Convert back to BGR for consistency
            self.show_image(self.processed_image)

    def apply_sepia(self):
        if self.processed_image is not None:
            self.save_state()
            img = np.array(self.processed_image, dtype=np.float64)
            img = cv2.transform(img, np.matrix([[0.272, 0.534, 0.131],
                                                [0.349, 0.686, 0.168],
                                                [0.393, 0.769, 0.189]]))
            img[np.where(img > 255)] = 255
            self.processed_image = np.array(img, dtype=np.uint8)
            self.show_image(self.processed_image)

    def apply_edge_detection(self):
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.Canny(self.processed_image, 100, 200)
            self.processed_image = cv2.cvtColor(self.processed_image, cv2.COLOR_GRAY2BGR)  # Convert back to BGR for consistency
            self.show_image(self.processed_image)

    def show_image(self, image):
        if image is not None and image.size != 0:
            # Resize the image to fit the canvas while maintaining the aspect ratio
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            img_height, img_width = image.shape[:2]

            scale = min(canvas_width / img_width, canvas_height / img_height)
            new_width = int(img_width * scale)
            new_height = int(img_height * scale)

            resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

            # Convert the image to RGB (OpenCV uses BGR by default)
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
            resized_image = Image.fromarray(resized_image)
            resized_image = ImageTk.PhotoImage(resized_image)

            self.canvas.delete("all")
            self.image_on_canvas = self.canvas.create_image(0, 0, anchor="nw", image=resized_image)
            self.canvas.image = resized_image

            self.canvas.config(scrollregion=self.canvas.bbox("all"))

    def close_editor(self):
        self.master.quit()

if __name__ == "__main__":
    root = Tk()
    editor = ImageEditor(root)
    root.mainloop()



SyntaxError: positional argument follows keyword argument (4118703226.py, line 46)

In [ ]:
import cv2
import numpy as np
from tkinter import Tk, Button, Scale, HORIZONTAL, filedialog, Toplevel, colorchooser, Canvas, Frame, Scrollbar, VERTICAL, RIGHT, LEFT, Y, BOTH, Label
from PIL import Image, ImageTk

class ImageEditor:
    def __init__(self, master):
        self.master = master
        self.master.title("簡易圖像編輯器")

        self.original_image = None  # 原始圖像
        self.processed_image = None  # 處理後的圖像
        self.image_states = []  # 圖像狀態堆疊
        self.undo_stack = []  # 撤銷堆疊

        self.drawing = False  # 畫筆狀態
        self.brush_size = 5  # 畫筆大小
        self.brush_color = (255, 0, 0)  # 畫筆顏色 (默認紅色)

        # 標題標籤
        self.title_label = Label(master, text="圖像編輯程式", font=("Arial", 25))
        self.title_label.pack()

        # 圖像框架和畫布
        self.frame = Frame(master, width=800, height=600)
        self.frame.pack(expand=True, fill=BOTH)

        self.canvas = Canvas(self.frame, bg='white')
        self.scroll_y = Scrollbar(self.frame, orient=VERTICAL, command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scroll_y.set)

        self.scroll_y.pack(side=RIGHT, fill=Y)
        self.canvas.pack(side=LEFT, expand=True, fill=BOTH)

        self.image_on_canvas = None  # 畫布上的圖像

        # 各種按鈕和控制
        self.load_button = Button(master, text="載入圖像", command=self.load_image)
        self.load_button.pack()

        self.invert_button = Button(master, text="反轉圖像", command=self.invert_image)
        self.invert_button.pack()

        self.brightness_scale = Scale(master, from_=-127, to=127, orient=HORIZONTAL, label="亮度", command=self.update_image)
        self.brightness_scale.pack()

        self.contrast_scale = Scale(master, from_=-127, to=127, orient=HORIZONTAL, label="對比度", command=self.update_image)
        self.contrast_scale.pack()

        self.saturation_scale = Scale(master, from_=-100, to=100, orient=HORIZONTAL, label="飽和度", command=self.update_image)
        self.saturation_scale.pack()

        self.crop_button = Button(master, text="裁剪圖像", command=self.start_crop)
        self.crop_button.pack()

        self.rotate_button = Button(master, text="旋轉圖像", command=self.rotate_image)
        self.rotate_button.pack()

        self.brush_button = Button(master, text="畫筆", command=self.prepare_brush)
        self.brush_button.pack()

        self.undo_button = Button(master, text="撤銷", command=self.undo)
        self.undo_button.pack()

        self.redo_button = Button(master, text="重做", command=self.redo)
        self.redo_button.pack()

        self.save_button = Button(master, text="保存圖像", command=self.save_image)
        self.save_button.pack()

        self.grayscale_button = Button(master, text="灰階", command=self.apply_grayscale)
        self.grayscale_button.pack()

        self.sepia_button = Button(master, text="復古", command=self.apply_sepia)
        self.sepia_button.pack()

        self.edge_button = Button(master, text="邊緣檢測", command=self.apply_edge_detection)
        self.edge_button.pack()

    def load_image(self):
        """載入圖像"""
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp")])
        if file_path:
            self.original_image = cv2.imread(file_path)
            if self.original_image is not None:
                self.processed_image = self.original_image.copy()
                self.image_states = [self.processed_image.copy()]
                self.undo_stack.clear()
                self.show_image(self.original_image)
            else:
                print("Failed to load image. Please select a valid image file.")

    def save_image(self):
        """保存圖像"""
        if self.processed_image is not None:
            file_path = filedialog.asksaveasfilename(defaultextension=".jpg",
                                                     filetypes=[("JPEG files", "*.jpg"),
                                                                ("PNG files", "*.png"),
                                                                ("BMP files", "*.bmp")])
            if file_path:
                cv2.imwrite(file_path, self.processed_image)

    def invert_image(self):
        """反轉圖像顏色"""
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.bitwise_not(self.processed_image)
            self.show_image(self.processed_image)

    def update_image(self, _=None):
        """更新圖像的亮度、對比度和飽和度"""
        if self.original_image is not None:
            brightness = self.brightness_scale.get()
            contrast = self.contrast_scale.get()
            saturation = self.saturation_scale.get()
            self.processed_image = self.adjust_brightness_contrast(self.original_image, brightness, contrast)
            self.processed_image = self.adjust_saturation(self.processed_image, saturation)
            self.show_image(self.processed_image)

    def adjust_brightness_contrast(self, image, brightness=0, contrast=0):
        """調整圖像的亮度和對比度"""
        brightness = np.clip(brightness, -127, 127)
        contrast = np.clip(contrast, -127, 127)

        B = brightness / 127.0
        C = contrast / 127.0

        k = np.tan((45 + 44 * C) / 180 * np.pi)

        img = image.astype(np.float32)
        img = (img - 127.5 * (1 - B)) * k + 127.5 * (1 + B)
        img = np.clip(img, 0, 255).astype(np.uint8)

        return img

    def adjust_saturation(self, image, saturation=0):
        """調整圖像的飽和度"""
        saturation = np.clip(saturation, -100, 100)
        if saturation == 0:
            return image

        img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
        img[:, :, 1] = img[:, :, 1] * (1 + saturation / 100.0)
        img[:, :, 1] = np.clip(img[:, :, 1], 0, 255)
        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_HSV2BGR)

        return img

    def start_crop(self):
        """開始裁剪圖像"""
        if self.processed_image is not None:
            self.cropping = True
            self.crop_rect = [0, 0, 0, 0]
            cv2.namedWindow("Crop Image")
            cv2.setMouseCallback("Crop Image", self.mouse_crop)
            while self.cropping:
                img_copy = self.processed_image.copy()
                if self.crop_rect[2] != 0 and self.crop_rect[3] != 0:
                    cv2.rectangle(img_copy, (self.crop_rect[0], self.crop_rect[1]), 
                                  (self.crop_rect[2], self.crop_rect[3]), (0, 255, 0), 2)
                cv2.imshow("Crop Image", img_copy)
                cv2.waitKey(1)
            cv2.destroyWindow("Crop Image")
            self.crop_image()

    def mouse_crop(self, event, x, y, flags, param):
        """處理裁剪時的滑鼠事件"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.crop_rect[0] = x
            self.crop_rect[1] = y
        elif event == cv2.EVENT_LBUTTONUP:
            self.crop_rect[2] = x
            self.crop_rect[3] = y
            self.cropping = False

    def crop_image(self):
        """裁剪圖像"""
        if self.processed_image is not None:
            x1, y1, x2, y2 = self.crop_rect
            if x1 < x2 and y1 < y2:
                self.save_state()
                self.processed_image = self.processed_image[y1:y2, x1:x2]
                self.show_image(self.processed_image)
            else:
                print("Invalid crop coordinates")

    def rotate_image(self):
        """旋轉圖像"""
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.rotate(self.processed_image, cv2.ROTATE_90_CLOCKWISE)
            self.show_image(self.processed_image)

    def prepare_brush(self):
        """準備畫筆工具"""
        if self.processed_image is not None:
            self.brush_window = Toplevel(self.master)
            self.brush_window.title("Brush Tool")

            self.brush_size_scale = Scale(self.brush_window, from_=1, to=50, orient=HORIZONTAL, label="Brush Size")
            self.brush_size_scale.pack()

            self.color_button = Button(self.brush_window, text="Choose Color", command=self.choose_color)
            self.color_button.pack()

            self.start_drawing_button = Button(self.brush_window, text="Start Drawing", command=self.start_brush)
            self.start_drawing_button.pack()

            self.brush_window.transient(self.master)
            self.brush_window.grab_set()

    def choose_color(self):
        """選擇畫筆顏色"""
        color_code = colorchooser.askcolor(title="選擇畫筆顏色")
        if color_code:
            self.brush_color = (int(color_code[0][2]), int(color_code[0][1]), int(color_code[0][0]))

    def start_brush(self):
        """開始使用畫筆"""
        self.brush_size = self.brush_size_scale.get()
        self.brush_window.destroy()

        if self.processed_image is not None:
            self.drawing = False

            cv2.namedWindow("Draw Image")
            cv2.setMouseCallback("Draw Image", self.mouse_draw)
            while True:
                cv2.imshow("Draw Image", self.processed_image)
                if cv2.waitKey(1) & 0xFF == 27:
                    break
            cv2.destroyWindow("Draw Image")
            self.save_state()
            self.show_image(self.processed_image)

    def mouse_draw(self, event, x, y, flags, param):
        """處理畫筆的滑鼠事件"""
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                cv2.circle(self.processed_image, (x, y), self.brush_size, self.brush_color, -1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False

    def save_state(self):
        """保存當前圖像狀態以支援撤銷和重做"""
        if self.processed_image is not None:
            self.image_states.append(self.processed_image.copy())
            self.undo_stack.clear()

    def undo(self):
        """撤銷上一個操作"""
        if len(self.image_states) > 1:
            self.undo_stack.append(self.image_states.pop())
            self.processed_image = self.image_states[-1]
            self.show_image(self.processed_image)

    def redo(self):
        """重做上一個撤銷的操作"""
        if self.undo_stack:
            self.image_states.append(self.undo_stack.pop())
            self.processed_image = self.image_states[-1]
            self.show_image(self.processed_image)

    def apply_grayscale(self):
        """應用灰階過濾器"""
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2GRAY)
            self.processed_image = cv2.cvtColor(self.processed_image, cv2.COLOR_GRAY2BGR)  # 轉回BGR以保持一致性
            self.show_image(self.processed_image)

    def apply_sepia(self):
        """應用復古過濾器"""
        if self.processed_image is not None:
            self.save_state()
            img = np.array(self.processed_image, dtype=np.float64)
            img = cv2.transform(img, np.matrix([[0.272, 0.534, 0.131],
                                                [0.349, 0.686, 0.168],
                                                [0.393, 0.769, 0.189]]))
            img[np.where(img > 255)] = 255
            self.processed_image = np.array(img, dtype=np.uint8)
            self.show_image(self.processed_image)

    def apply_edge_detection(self):
        """應用邊緣檢測過濾器"""
        if self.processed_image is not None:
            self.save_state()
            self.processed_image = cv2.Canny(self.processed_image, 100, 200)
            self.processed_image = cv2.cvtcolor(self.processed_image, cv2.COLOR_GRAY2BGR)  # 轉回BGR以保持一致性
            self.show_image(self.processed_image)

    def show_image(self, image):
        """在畫布上顯示圖像"""
        if image is not None and image.size != 0:
            # 調整圖像大小以適應畫布，同時保持長寬比
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            img_height, img_width = image.shape[:2]

            scale = min(canvas_width / img_width, canvas_height / img_height)
            new_width = int(img_width * scale)
            new_height = int(img_height * scale)

            resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

            # 將圖像轉換為RGB格式 (OpenCV默認使用BGR)
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
            resized_image = Image.fromarray(resized_image)
            resized_image = ImageTk.PhotoImage(resized_image)

            self.canvas.delete("all")
            self.image_on_canvas = self.canvas.create_image(0, 0, anchor="nw", image=resized_image)
            self.canvas.image = resized_image

            self.canvas.config(scrollregion=self.canvas.bbox("all"))

    def close_editor(self):
        """關閉編輯器"""
        self.master.quit()

if __name__ == "__main__":
    root = Tk()
    editor = ImageEditor(root)
    root.mainloop()
